# Phase 2: Structural Pattern Analysis

**Question:** What structural properties distinguish hallucination graphs from truthful graphs?

This notebook:
1. Computes topology features (diameter, clustering, components)
2. Analyzes alignment-specific patterns (alignment degree, flow distributions)
3. Performs spectral analysis (Laplacian eigenvalues, spectral gap)
4. Runs community detection (Louvain) and analyzes structure differences

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from calamr_interp.utils.data_loading import load_dataset
from calamr_interp.utils.visualization import setup_style, violin_comparison, COLORS, LABEL_NAMES
from calamr_interp.utils.statistics import mann_whitney_test, cohens_d, bonferroni_correction
from calamr_interp.phase2_structural import StructuralAnalyzer

setup_style()
print("Imports OK")

## 1. Load Data & Extract Structural Features

In [ ]:
dataset = load_dataset()
print(f"Loaded {len(dataset)} graphs")

analyzer = StructuralAnalyzer()
struct_df, labels = analyzer.extract_batch(dataset)
print(f"Extracted {len(struct_df.columns)} structural features")
struct_df.describe()

## 2. Topology Comparison

In [ ]:
# Topology features
topo_features = ['density', 'n_connected_components', 'avg_clustering',
                 'avg_degree', 'degree_std', 'max_degree', 'diameter', 'avg_shortest_path']
topo_features = [f for f in topo_features if f in struct_df.columns]

data_dict = {}
for col in topo_features:
    data_dict[col] = {
        0: struct_df.loc[labels == 0, col].values,
        1: struct_df.loc[labels == 1, col].values,
    }

fig = violin_comparison(data_dict, title='Topology Features: Truth vs Hallucination', ncols=4)
plt.show()

In [ ]:
# Statistical tests for all structural features
test_results = []
for col in struct_df.columns:
    truth_vals = struct_df.loc[labels == 0, col].values
    hallu_vals = struct_df.loc[labels == 1, col].values
    mw = mann_whitney_test(truth_vals, hallu_vals)
    d = cohens_d(hallu_vals, truth_vals)
    test_results.append({
        'feature': col,
        'truth_mean': truth_vals.mean(),
        'hallu_mean': hallu_vals.mean(),
        'p_value': mw['p_value'],
        'effect_size': mw['effect_size_r'],
        'cohens_d': d,
    })

test_df = pd.DataFrame(test_results).sort_values('p_value')

# Bonferroni correction
test_df['p_corrected'] = bonferroni_correction(test_df['p_value'].tolist())
test_df['significant'] = test_df['p_corrected'] < 0.05
print(f"{test_df['significant'].sum()} features significant after Bonferroni correction")
test_df

## 3. Alignment-Specific Patterns

In [ ]:
# Alignment pattern features
align_features = [f for f in struct_df.columns if 'flow' in f or 'alignment' in f or 'aligned' in f]

data_dict = {}
for col in align_features:
    data_dict[col] = {
        0: struct_df.loc[labels == 0, col].values,
        1: struct_df.loc[labels == 1, col].values,
    }

fig = violin_comparison(data_dict, title='Alignment Patterns: Truth vs Hallucination', ncols=3)
plt.show()

In [ ]:
# Flow distribution CDFs
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, feat in enumerate(['flow_mean', 'flow_nonzero_fraction']):
    if feat not in struct_df.columns:
        continue
    ax = axes[i]
    for label, name, color in [(0, 'Truth', COLORS['truth']), (1, 'Hallu', COLORS['hallu'])]:
        vals = np.sort(struct_df.loc[labels == label, feat].values)
        cdf = np.arange(1, len(vals) + 1) / len(vals)
        ax.plot(vals, cdf, color=color, label=name, linewidth=2)
    ax.set_title(f'CDF: {feat}')
    ax.set_xlabel(feat)
    ax.set_ylabel('CDF')
    ax.legend()

plt.tight_layout()
plt.show()

## 4. Spectral Analysis

In [ ]:
# Spectral features
spectral_features = [f for f in struct_df.columns if 'spectral' in f or 'eigenvalue' in f or 'algebraic' in f]

if spectral_features:
    data_dict = {}
    for col in spectral_features:
        data_dict[col] = {
            0: struct_df.loc[labels == 0, col].values,
            1: struct_df.loc[labels == 1, col].values,
        }
    fig = violin_comparison(data_dict, title='Spectral Features', ncols=2)
    plt.show()
else:
    print('No spectral features found')

## 5. Community Structure

In [ ]:
# Community features
comm_features = [f for f in struct_df.columns if 'communit' in f or 'modularity' in f]

if comm_features:
    data_dict = {}
    for col in comm_features:
        data_dict[col] = {
            0: struct_df.loc[labels == 0, col].values,
            1: struct_df.loc[labels == 1, col].values,
        }
    fig = violin_comparison(data_dict, title='Community Structure', ncols=3)
    plt.show()
else:
    print('No community features found')

In [ ]:
# Summary: top discriminative structural features
print("\n=== Top Discriminative Structural Features ===")
print("(sorted by Mann-Whitney p-value, Bonferroni corrected)\n")
top = test_df[test_df['significant']].head(10)
if len(top) > 0:
    print(top[['feature', 'truth_mean', 'hallu_mean', 'cohens_d', 'p_corrected']].to_string(index=False))
else:
    print("No features significant after Bonferroni correction.")
    print("\nTop 5 by uncorrected p-value:")
    print(test_df.head(5)[['feature', 'truth_mean', 'hallu_mean', 'cohens_d', 'p_value']].to_string(index=False))

print("\n--- Key Takeaway ---")
print("If flow-related features (flow_mean, flow_nonzero_fraction) show significant")
print("differences, it confirms that hallucinated summaries have weaker alignment connectivity.")